In [153]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV
from time import time
import numpy as np
import warnings
from nltk import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
import nltk
from nltk.corpus import wordnet
nltk.download("wordnet")
nltk.download("averaged_perceptron_tagger")
import functools
import math
import pickle
from sklearn.externals import joblib

[nltk_data] Downloading package wordnet to /home/ubuntu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/ubuntu/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
movies_df = pd.read_csv('joined_movies_w_factors_and_genomes.csv')

In [197]:
# descriptive
print(movies_df.columns.values[0:100])
print(movies_df.dtypes)
movies_df

['index' 'title_script' 'genre_script' 'filename' 'title_p' 'movieId'
 'title' 'genres' 'X1' 'X2' 'X3' 'X4' 'X5' 'X6' 'X7' 'X8' 'X9' 'X10' 'X11'
 'X12' 'X13' 'X14' 'X15' 'X16' 'X17' 'X18' 'X19' 'X20' 'X21' 'X22' 'X23'
 'X24' 'X25' 'X26' 'X27' 'X28' 'X29' 'X30' 'G1' 'G2' 'G3' 'G4' 'G5' 'G6'
 'G7' 'G8' 'G9' 'G10' 'G11' 'G12' 'G13' 'G14' 'G15' 'G16' 'G17' 'G18' 'G19'
 'G20' 'G21' 'G22' 'G23' 'G24' 'G25' 'G26' 'G27' 'G28' 'G29' 'G30' 'G31'
 'G32' 'G33' 'G34' 'G35' 'G36' 'G37' 'G38' 'G39' 'G40' 'G41' 'G42' 'G43'
 'G44' 'G45' 'G46' 'G47' 'G48' 'G49' 'G50' 'G51' 'G52' 'G53' 'G54' 'G55'
 'G56' 'G57' 'G58' 'G59' 'G60' 'G61' 'G62']
index             int64
title_script     object
genre_script     object
filename         object
title_p          object
movieId           int64
title            object
genres           object
X1              float64
X2              float64
X3              float64
X4              float64
X5              float64
X6              float64
X7              float64
X8        

,index,title_script,genre_script,filename,title_p,movieId,title,genres,X1,X2,...,G1119,G1120,G1121,G1122,G1123,G1124,G1125,G1126,G1127,G1128
0,0,10 Things I Hate About You,"Comedy,Romance",10ThingsIHateAboutYou.txt,10 things i hate about you,2572,10 Things I Hate About You (1999),Comedy|Romance,-0.016455,0.018789,...,0.03600,0.02150,0.02275,0.01975,0.26425,0.09425,0.01425,0.01475,0.08425,0.01900
1,1,12 Years a Slave,Drama,12YearsaSlave.txt,12 years a slave,105844,12 Years a Slave (2013),Drama,0.008645,0.003144,...,0.07650,0.04550,0.08550,0.06875,0.16950,0.11200,0.02500,0.05175,0.09950,0.02600
2,2,127 Hours,"Adventure,Drama,Thriller",127Hours.txt,127 hours,81562,127 Hours (2010),Adventure|Drama|Thriller,0.012059,-0.001237,...,0.06000,0.02250,0.01625,0.11225,0.16725,0.10250,0.02800,0.01700,0.18200,0.03775
3,3,1492: Conquest of Paradise,"Adventure,Drama",1492ConquestofParadise.txt,1492: conquest of paradise,8905,1492: Conquest of Paradise (1992),Adventure|Drama,-0.001974,-0.005688,...,0.09000,0.04075,0.02200,0.02625,0.21350,0.05575,0.02325,0.02800,0.07675,0.01575
4,4,15 Minutes,"Action,Crime,Thriller",15Minutes.txt,15 minutes,4167,15 Minutes (2001),Thriller,-0.011999,-0.005283,...,0.04275,0.01425,0.01225,0.03300,0.33225,0.04800,0.02475,0.01300,0.10300,0.01825
5,5,17 Again,"Comedy,Drama,Romance",17Again.txt,17 again,68135,17 Again (2009),Comedy|Drama,-0.009621,0.009796,...,0.02575,0.01700,0.01575,0.01975,0.23350,0.05800,0.01500,0.00725,0.05775,0.01350
6,6,187,Drama,187.txt,187,1609,187 (One Eight Seven) (1997),Drama|Thriller,-0.004879,-0.009280,...,0.04350,0.01600,0.00975,0.05325,0.23750,0.08475,0.02025,0.00900,0.07475,0.01625
7,7,2001: A Space Odyssey,"Adventure,Sci-Fi",2001ASpaceOdyssey.txt,2001: a space odyssey,924,2001: A Space Odyssey (1968),Adventure|Drama|Sci-Fi,0.040611,-0.009755,...,0.04100,0.02650,0.02225,0.07375,0.16975,0.10350,0.08850,0.01375,0.05700,0.01875
8,8,2012,"Action,Adventure,Drama,Sci-Fi,Thriller",2012.txt,2012,72378,2012 (2009),Action|Drama|Sci-Fi|Thriller,-0.021496,0.002300,...,0.04725,0.01900,0.01825,0.01425,0.26750,0.08300,0.02050,0.01175,0.10775,0.01800
9,9,25th Hour,"Crime,Drama",25thHour.txt,25th hour,5954,25th Hour (2002),Crime|Drama,0.020300,-0.008028,...,0.10450,0.02650,0.01925,0.17400,0.33475,0.21325,0.03325,0.01850,0.06600,0.02000


For some reason there are still movies without scripts and movies with NA's (check why the NAs are there!!!). Remove them here.

In [3]:
#Missing file names
movies_df = movies_df[pd.notnull(movies_df["filename"])]
#Remove pdfs
movies_df = movies_df[movies_df["filename"].str.contains(r"\.txt$")]
#Manually remove these two since the scripts are missing
movies_df = movies_df[(movies_df["filename"] != "Apollo13.txt") & (movies_df["filename"] != "ScaryMovie2.txt")]
#NaN factors
movies_df = movies_df[pd.notnull(movies_df["X1"])]
#by removing movies with NaN genome, we are remove some movies (over a dozen)
#that would be good for regressing against factors X1
movies_df = movies_df[pd.notnull(movies_df["G1"])]
movies_df.reset_index(inplace=True)

In [4]:
script_files = ('raw/' + movies_df.filename).tolist()

Some stuff on CountVectorizer from the documentation:

"Convert a collection of text documents to a matrix of token counts"

Makes everything lowercase by default.

Default tokenizer: "The default regexp select tokens of 2 or more alphanumeric characters (punctuation is completely ignored and always treated as a token separator)."

Can use a built-in stop word list for English.

It cannot be parallelized, but you can use HashingVectorizer instead, which has some limitations (can't figure out what words are associated with each feature?). CountVectorizer is fast enough for our ~800 scripts.

In [5]:
# We want to do some stemming before building the bag-of-words.
# We do this by building a new class that inherits from CountVectorizer.
## This is the simplest thing to do, as it allows us to use all the nice
## features from CountVectorizer (preprocessing to remove uppercase and 
## punctuation, decent tokenizing, stop words) and just add the stemming
## step, rather than having to rewrite everything.

class StemmedCountVectorizer(CountVectorizer):
    def __init__(self, stemmer, *args, **kwargs):
        super(StemmedCountVectorizer,self).__init__(*args, **kwargs)
        self.stemmer = stemmer
    
    def build_analyzer(self):
        analyzer = super(StemmedCountVectorizer,self).build_analyzer()
        return lambda doc:[self.stemmer.stem(w) for w in analyzer(doc)]
    
class LemmaCountVectorizer(CountVectorizer):
    def __init__(self, *args, **kwargs):
        super(LemmaCountVectorizer,self).__init__(*args, **kwargs)
        self.wnl = WordNetLemmatizer()
        
    #the lemmatizer always needs a part of speech
    #use pos_tag to get the part of speech and then lemmatize
    #memoize this since pos_tag is very sloow
    @functools.lru_cache(maxsize=100000)
    def pos_tag_and_lemmatize(self,word):
        pos_tag = nltk.pos_tag([word])
        pos = get_wordnet_pos(pos_tag[0][1])
        return self.wnl.lemmatize(word,pos)
    
    def build_analyzer(self):
        analyzer = super(LemmaCountVectorizer,self).build_analyzer()
        return lambda doc:[self.pos_tag_and_lemmatize(w) for w in analyzer(doc)]

#The parts of speech returned by pos_tag do not match the ones
#from wordnet. Convert them.
#This could probably be a lot more efficient.
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [6]:
#vectorizer = StemmedCountVectorizer(input='filename',decode_error='ignore',stop_words='english', max_features=10000, stemmer=PorterStemmer())
vectorizer = LemmaCountVectorizer(input='filename',decode_error='ignore',stop_words='english', max_features=10000)

In [7]:
start = time()
features = vectorizer.fit_transform(script_files)
print(time()-start)

43.851587533950806


In [8]:
print(vectorizer.vocabulary_["great"])
print(movies_df.ix[484]["title"])
#great is 3993
#movie 484 is man on the moon
print(features[484,4033])
#we can verify that man of the moon has 17 "great"s

4033
Man on the Moon (1999)
18


In [79]:
def random_forest_grid_search(target, features):
    forest = RandomForestRegressor(n_jobs=-1)
    
    n_features = features.get_shape()[1]
    max_features = [math.floor(math.sqrt(n_features)),n_features//3]
    if n_features < 4000:
        max_features.append(n_features)

    param_grid = {"max_features": max_features,
                  "max_depth": [None],
                  "n_estimators": [200,500,1000]}

    #3-fold by default
    grid_search = GridSearchCV(forest,param_grid=param_grid)
    start = time()
    grid_search.fit(features,target)
    elapsed = time() - start
    print("Time to fit: %f" % elapsed)
    return grid_search

def print_importances(rf_model,n = 25):
    importances = rf_model.feature_importances_
    indices = np.argsort(importances)[::-1] #[::-1] just reverses
    
    print("Feature ranking:")
    for f in range(25):
        print("%d. %s (%f)" % (f+1,vectorizer.get_feature_names()[indices[f]], importances[indices[f]]))

def elastic_net_grid_search(target, features):
    elastic = ElasticNet(normalize=True)

    param_grid_el = {"l1_ratio": [0.0,0.1,0.3,0.5,0.7,0.9,1.0],
                     "alpha": [1,1e-1,1e-2,1e-3,1e-4,1e-5]}

    grid_search_el = GridSearchCV(elastic,param_grid=param_grid_el,cv=10,n_jobs=36)
    start = time()
    # We will often get a warning that the fit did not converge,
    # because many of the grid values may be bad
    with warnings.catch_warnings():
        grid_search_el.fit(features,target)
    print(time() - start)
    return grid_search_el

# First run a random forest regression over all features
# to find the N most important features.
## Then run both elastic net and random forest fits using only
## the smaller subset of N features. The smaller random forest
## fits will be the best predictors. The elastic net fits are used
## to distinguish between "positive" features (which predict large or
## positve target) and "negative" features (small/negative target).
### From experimentation (though only with one particular target),
### it seems like 20-100 are decent values for N (they produce the best
### scores on the limited feature fits). 100 was the best, but 20 and 50
### were also decent. 10 and 200 were significantly worse.
#### This returns (the large random forest grid, the feature names
####               sorted by importance of the best large random forest,
####               an array of smaller random forest grids (one for each value of N),
####               an array of smaller elastic net grids)
def elastic_forest(target):
    rf_grid_all = random_forest_grid_search(target, features)

    importances = rf_grid_all.best_estimator_.feature_importances_
    ordered_indices = np.argsort(importances)[::-1] #[::-1] just reverses
    feature_names = [vectorizer.get_feature_names()[i] for i in ordered_indices]
    
    rf_grids = []
    en_grids = []
    for n_features in [20,50,100,200]:
        rf_grids.append(random_forest_grid_search(target,features[:,ordered_indices[0:n_features]]))
        en_grids.append(elastic_net_grid_search(target, features[:,ordered_indices[0:n_features]]))
    
    return (rf_grid_all,feature_names,rf_grids,en_grids)

In [186]:
# In sklearn 0.18, there is a bug that prevents pickling of GridSearchCV
# objects. This is because they used masked numpy arrays, which pickle
# does not handle. Make a work around to convert all masked arrays to
# unmasked, before pickling

def process_GridSearchCV_pickling(grid_search_obj):
    for k,v in grid_search_obj.cv_results_.items():
        if not k.startswith("param_"):
            continue
        #convert masked array to array
        grid_search_obj.cv_results_[k] = np.array(v)

def process_fit_object_pickling(fit_object):
    process_GridSearchCV_pickling(fit_object[0])
    for fit in fit_object[2]:
        process_GridSearchCV_pickling(fit)
    for fit in fit_object[3]:
        process_GridSearchCV_pickling(fit)

In [ ]:
targets = ["X" + str(i) for i in range(1,31)]
fits = []

for target in targets:
    fit = elastic_forest(movies_df[target])
    fits.append(fit)
    process_fit_object_pickling(fit)
    with open("saved_objects/"+target+"_fit.pickle","wb") as handle:
        pickle.dump(fit,handle)

In [188]:
x4_fit = elastic_forest(movies_df["X2"])

Time to fit: 147.014942
Time to fit: 32.715335


/home/ubuntu/venv/lib/python3.5/site-packages/sklearn/linear_model/coordinate_descent.py:479: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/ubuntu/venv/lib/python3.5/site-packages/sklearn/linear_model/coordinate_descent.py:479: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/ubuntu/venv/lib/python3.5/site-packages/sklearn/linear_model/coordinate_descent.py:479: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/ubuntu/venv/lib/python3.5/site-packages/sklearn/linear_model/coordinate_descent.py:479: ConvergenceWarning: Objective did not converge. You might wa

2.6552932262420654
Time to fit: 36.015357


/home/ubuntu/venv/lib/python3.5/site-packages/sklearn/linear_model/coordinate_descent.py:479: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/ubuntu/venv/lib/python3.5/site-packages/sklearn/linear_model/coordinate_descent.py:479: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/ubuntu/venv/lib/python3.5/site-packages/sklearn/linear_model/coordinate_descent.py:479: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/ubuntu/venv/lib/python3.5/site-packages/sklearn/linear_model/coordinate_descent.py:479: ConvergenceWarning: Objective did not converge. You might wa

3.1547577381134033
Time to fit: 39.457095


/home/ubuntu/venv/lib/python3.5/site-packages/sklearn/linear_model/coordinate_descent.py:479: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/ubuntu/venv/lib/python3.5/site-packages/sklearn/linear_model/coordinate_descent.py:479: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/ubuntu/venv/lib/python3.5/site-packages/sklearn/linear_model/coordinate_descent.py:479: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/ubuntu/venv/lib/python3.5/site-packages/sklearn/linear_model/coordinate_descent.py:479: ConvergenceWarning: Objective did not converge. You might wa

4.1008055210113525
Time to fit: 48.193764


/home/ubuntu/venv/lib/python3.5/site-packages/sklearn/linear_model/coordinate_descent.py:479: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/ubuntu/venv/lib/python3.5/site-packages/sklearn/linear_model/coordinate_descent.py:479: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/ubuntu/venv/lib/python3.5/site-packages/sklearn/linear_model/coordinate_descent.py:479: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/ubuntu/venv/lib/python3.5/site-packages/sklearn/linear_model/coordinate_descent.py:479: ConvergenceWarning: Objective did not converge. You might wa

5.546707391738892


/home/ubuntu/venv/lib/python3.5/site-packages/sklearn/linear_model/coordinate_descent.py:479: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [189]:
process_fit_object_pickling(x4_fit)
with open("saved_objects/test_x4_fit.pickle","wb") as handle:
    pickle.dump(x4_fit,handle)

In [194]:
with open("saved_objects/test_x4_fit.pickle","rb") as handle:
    test = pickle.load(handle)

/home/ubuntu/venv/lib/python3.5/site-packages/sklearn/model_selection/_search.py:662: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: -0.00777, std: 0.01223, params: {'alpha': 1, 'l1_ratio': 0.0},
 mean: -0.01075, std: 0.01186, params: {'alpha': 1, 'l1_ratio': 0.1},
 mean: -0.01075, std: 0.01186, params: {'alpha': 1, 'l1_ratio': 0.3},
 mean: -0.01075, std: 0.01186, params: {'alpha': 1, 'l1_ratio': 0.5},
 mean: -0.01075, std: 0.01186, params: {'alpha': 1, 'l1_ratio': 0.7},
 mean: -0.01075, std: 0.01186, params: {'alpha': 1, 'l1_ratio': 0.9},
 mean: -0.01075, std: 0.01186, params: {'alpha': 1, 'l1_ratio': 1.0},
 mean: 0.01713, std: 0.01716, params: {'alpha': 0.1, 'l1_ratio': 0.0},
 mean: -0.01075, std: 0.01186, params: {'alpha': 0.1, 'l1_ratio': 0.1},
 mean: -0.01075, std: 0.01186, params: {'alpha': 0.1, 'l1_ratio': 0.3},
 mean: -0.01075, std: 0.01186, params: {'alpha': 0.1, 'l1_ratio': 0.5},
 mean: -0.01075, std: 0.01186, params: {'alpha': 0.1, 'l1_ratio': 0.7},
 mean: -0.01075, std: 0.01186, params: {'alpha': 0.1, 'l1_ratio': 0.9},
 mean: -0.01075, std: 0.01186, params: {'alpha': 0.1, 'l1_ratio': 1.0},
 mean: 

In [109]:
#How to get importances/coefficients
#for the full random forest (this only works for the best estimator)
dict(zip(x4_fit[1],np.sort(x4_fit[0].best_estimator_.feature_importances_,)[::-1]))
#one of the random forests with fewer features
dict(zip(x4_fit[1][0:50],x4_fit[2][1].best_estimator_.feature_importances_))
#one of the elastic nets with fewer features
dict(zip(x4_fit[1][0:50],x4_fit[3][1].best_estimator_.coef_))

{'arm': -0.0,
 'ask': 2.4273075933105468e-06,
 'bitch': -0.0,
 'blood': -4.7081119163236433e-05,
 'body': -6.3883066128336488e-05,
 'box': 0.0,
 'broom': 0.0,
 'clothes': 0.0,
 'come': 1.7320106461579729e-05,
 'day': 0.0,
 'dead': -0.0,
 'denny': -0.00028346633159504619,
 'door': -0.0,
 'excuse': 0.0,
 'flesh': -0.0002981793977530888,
 'fuck': -0.00010193992493098029,
 'fucker': -4.4706060132273728e-05,
 'fuckin': -4.2015943882449404e-05,
 'gun': -1.5527972053474235e-05,
 'hand': -0.0,
 'hey': 0.0,
 'ho': 0.0,
 'hoo': 0.00065399276111242303,
 'human': -0.0,
 'int': 0.0,
 'lime': -0.00019985497363535955,
 'll': 0.0,
 'marry': 0.00073399178336144301,
 'mayhem': -0.0,
 'motherfucker': -0.0002674387456674509,
 'nod': 3.3657358817495898e-05,
 'oh': 0.0,
 'personal': 0.0,
 'potion': 0.0,
 'potter': 0.00012133057078376908,
 'pussy': -0.00054995799789198476,
 'say': 0.0,
 'scream': -0.00014539217320013359,
 'shit': -0.0,
 'sir': 1.3846686056723491e-06,
 'spell': 0.0008862087820058844,
 'tail':

In [127]:
x4_fit[3][0].best_score_

0.20173182885386404

In [6]:
movies_df.sort_values(by="X2")[["title","X2"]]

,title,X2
536,Natural Born Killers (1994),-0.075095
282,Fight Club (1999),-0.064130
298,From Dusk Till Dawn (1996),-0.060262
277,Fear and Loathing in Las Vegas (1998),-0.059811
267,Event Horizon (1997),-0.059577
763,Trainspotting (1996),-0.052934
108,"Big Lebowski, The (1998)",-0.052570
604,Pulp Fiction (1994),-0.051385
47,American Psycho (2000),-0.051318
624,Reservoir Dogs (1992),-0.050662
